# 📊  Example of interactive plot

This Jupyter notebook compute a mini example of what output figure to expect when running the pipeline. 

This figure is generated from random data.

In [1]:
from cwas_rsfmri.files import verify_atlas_files

import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go


In [3]:
output_path = "../../src/cwas_rsfmri/tests/data/bids"
pvalues_matrix_file = os.path.join(output_path, "output",
                                 "cwas_NDD_HC_rsfmri_denoiseSimple_example_atlas_fdr_corrected_pvalues.tsv")
beta_matrix_file = os.path.join(output_path, "output",
                                    "cwas_NDD_HC_rsfmri_denoiseSimple_example_atlas_standardized_betas.tsv")

atlas_file = os.path.join(output_path, "example_atlas.tsv")
_, labels = verify_atlas_files(atlas_file)

⏳ Verifying altas location ...
path to access atlas: ../../src/cwas_rsfmri/tests/data/bids/example_atlas.tsv


In [4]:
beta_matrix = pd.read_csv(beta_matrix_file, sep="\t", index_col=0)
pvalues_matrix = pd.read_csv(pvalues_matrix_file, sep="\t", index_col=0)

In [5]:
print("\n📊 Plotting interactive connectivity matrix ...\n")

# Prepare pvalues matrix
pvalues_corrected = np.array(pvalues_matrix)
significance_mask = np.zeros_like(pvalues_corrected, dtype=bool)
upper_triangle_indices = np.triu_indices_from(pvalues_corrected, k=1)
significance_mask[upper_triangle_indices] = pvalues_corrected[upper_triangle_indices] < 0.05

# Prepare beta matrix
beta_matrix = np.array(beta_matrix)
abs_max = np.nanmax(np.abs(beta_matrix))
zmin, zmax = -abs_max, abs_max

# Create a full matrix of NaNs with same shape
beta_matrix_upper = np.full_like(beta_matrix, np.nan)

# Fill in only the upper triangle (excluding diagonal)
i_upper, j_upper = np.triu_indices_from(beta_matrix, k=1)
beta_matrix_upper[i_upper, j_upper] = beta_matrix[i_upper, j_upper]

custom_colorscale = [
    [0.0, "blue"],   
    [0.5, "white"], 
    [1.0, "red"]     
]

# Prepare beta heatmap
heatmap = go.Heatmap(
    z=beta_matrix_upper,
    x=labels,
    y=labels,
    colorscale=custom_colorscale,
    zmin=zmin,
    zmax=zmax,
    colorbar=dict(title='Beta values'),
    hovertemplate="%{y} - %{x}<br>Beta value: %{z}<extra></extra>"
)

# Semi-transparent mask to indicate significance
mask = np.where(significance_mask, 1.0, np.nan)  # show only where significant
sig_overlay = go.Heatmap(
    z=mask,
    x=labels,
    y=labels,
    showscale=False,
    colorscale=[[0, 'rgba(255,255,0,0.6)'], [1, 'rgba(255,255,0,0.6)']],  # with yellow overlay
    hoverinfo='skip'
)

# Combine
fig = go.Figure(data=[heatmap, sig_overlay])
fig.update_layout(
    title=f'Interactive beta value heatmap with FDR correction',
    xaxis=dict(title='Labels (Colonnes)', tickangle=45),
    yaxis=dict(title='Labels (Lignes)', autorange='reversed'),
    width=600,  
    height=400   
)
fig


📊 Plotting interactive connectivity matrix ...

